In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

interesting = """784_OPPORTUNITY_S4-ADL5_1100_1200_col_id_17_1100_1200   -1.0
789_OPPORTUNITY_S3-ADL5_1477_1577_col_id_5_1477_1577    -0.9
799_OPPORTUNITY_S3-ADL4_895_995_col_id_49_895_995       -0.9
796_OPPORTUNITY_S4-ADL2_2085_2185_col_id_17_2085_2185   -0.9
781_OPPORTUNITY_S4-ADL2_2085_2185_col_id_37_2085_2185   -0.9
057_UCR_Anomaly_InternalBleeding14_1875_5607             0.9
237_Yahoo_A3Benchmark-TS22_data_500_730                  1.0
256_Yahoo_A4Benchmark-TS66_data_500_202                  1.0
247_Yahoo_A3Benchmark-TS38_data_500_151                  1.0
265_Yahoo_A4Benchmark-TS87_data_500_1616                 1.0""".split("\n")
interesting_datasets = [x.split(" ")[0] for x in interesting]

In [ ]:
def load_series(files):
    results = {}
    for f in files:
        df = pd.read_csv("benchmark_exp/TSB-AD/TSB-AD-U/" + f + ".csv")
        results[f.split(".")[0]] = df
    return results

res = load_series(interesting_datasets)

In [ ]:
seasonal_decompose = pd.read_csv("results_seasonal_decompose.csv", index_col=0)
seasonal_decompose['algorithm'] = "seasonal decompose"
seasonal_decompose = seasonal_decompose.rename(columns={'name': 'dataset'})

In [ ]:
df = pd.read_parquet("dd2ab5d8-43f0-4a58-acf4-70285684f4be_0_49abff8e02de4e998b1ab9a30f3d7878.parquet").set_index("DataSetName")

def get_score_sd(dataset):
    score = df.loc[dataset, 'series_decompose_anomalies_Values_ad_score']
    return np.abs(pd.Series(score[1:-1].split(b",")).astype(float))

In [ ]:
def get_score(method, dataset):
    return np.load(f"benchmark_exp/score/uni/{method}/{d}.npy")

In [ ]:
from TSB_AD.utils.slidingWindows import find_length_rank
from utils import plot_detection
from sklearn.metrics import average_precision_score
import numpy as np

for d in interesting_datasets:
    score_SR = get_score("Decompose", d)
    window_length = find_length_rank(res[d]['Data'])
    labels = res[d]['Label']
    plot_detection(res[d]['Data'], labels, scores=score_SR, window_length=window_length)
    plt.title(f"Decompose {d} : {average_precision_score(labels, score_SR):.2f}  ({window_length})")
    score_SD = get_score_sd(d)
    plot_detection(res[d]['Data'], labels, scores=score_SD, window_length=window_length)
    plt.title(f"Seasonal Decompose {d} :{average_precision_score(labels, score_SD):.2f} ({window_length})")